In [31]:
!pip install transformers==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 563 kB 37.4 MB/s 
     |████████████████████████████████| 1.3 MB 61.9 MB/s 
     |████████████████████████████████| 132 kB 68.3 MB/s 
     |████████████████████████████████| 880 kB 63.1 MB/s 
     |████████████████████████████████| 5.6 MB 51.3 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 9.1 MB 34.6 MB/s 
     |████████████████████████████████| 140 kB 64.3 MB/s 
     |████████████████████████████████| 127 kB 86.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e6b64fc39c94c56fcce6de77518219998c08e9044b4f09d45c2145bf568f922a
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    

In [1]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 32.3 MB/s 


In [28]:
import pandas as pd
import  glob 
import os 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential 
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten
from keras.utils.np_utils import to_categorical 
from keras.callbacks import EarlyStopping 
from sklearn.model_selection import train_test_split
from keras.layers import Dropout 
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D

In [3]:
#Loading and annotating the data
sent_analyzer = SentimentIntensityAnalyzer()
path = r'/content/drive/MyDrive/inputs'
raw_files = glob.glob(os.path.join(path,  "*.csv"))
dfs_list = []
for file in raw_files: 
    df = pd.read_csv(file)
    dfs_list.append(df)
all_tweets_df = pd.concat(dfs_list, axis=0, ignore_index=True)
#print(all_tweets_df.head(2))
all_tweets_df['compound_score'] = all_tweets_df['clean_tweets'].apply(lambda x: sent_analyzer.polarity_scores(str(x))['compound'])
all_tweets_df['label'] = all_tweets_df['compound_score'].apply(lambda x: "Positive" if x > 0.0 else "Negative" if x < 0.0 else "Neutral")
all_tweets_df = all_tweets_df.sample(frac=1)

In [4]:
#results
all_tweets_df.head(2)

,Date,User,Tweet,clean_tweets,compound_score,label
11067,2022-09-04 14:36:41+00:00,たんぽぽ組まなみん🍀,@UmaiyoUmesyu @ORAL_UVER_APPLE めっちゃ並んで食べたけど、これ...,めっちゃ並んで食べたけど、これは並んでも食べるね！って思うくらいには、美味しかった！！,0.0,Neutral
8033,2022-08-30 22:34:31+00:00,Nyakwar Dana,@KeEquityBank @MatiangDr @jumaf3 Check DM. I n...,Check DM. I need assistance ASAP.,0.0,Neutral


In [5]:
#declaring vocabulary parameters.
MAX_WORDS =  5000
max_length = 150 
embed_dim = 150 

In [13]:
#The textual features in the data will be converted to tokens using the tokenizer function in lstms as demonstrated belw:
tok = Tokenizer(num_words = MAX_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True )
all_tweets_df['clean_tweets'] = all_tweets_df['clean_tweets'].astype(str)
tok.fit_on_texts(all_tweets_df['clean_tweets'].values)
word_index = tok.word_index
print("Number of unique tokens: " + str(len(word_index)))

Number of unique tokens: 25964


In [14]:
X = tok.texts_to_sequences(all_tweets_df['clean_tweets'].values)
X = pad_sequences(X, maxlen = max_length )# since tweets have a limit of 140 characters, we use this figure as our max. length as defined before
print("Input tensor shape: ", X.shape)

Input tensor shape:  (16118, 150)


In [17]:
y = pd.get_dummies(all_tweets_df['label']).values

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
#initializing neural network
lstm_model = Sequential()
lstm_model.add(Embedding(MAX_WORDS, embed_dim, input_length = X.shape[1]))
lstm_model.add(SpatialDropout1D(0.2))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation="softmax"))
lstm_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
#setting model runtime params and fitting model
epochs = 10
batch_size = 32
lstm_history = lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[EarlyStopping(monitor="val_loss", patience=5, min_delta=0.0001 )])

Epoch 1/10
318/318 [==============================] - 80s 243ms/step - loss: 0.6780 - accuracy: 0.7220 - val_loss: 0.5192 - val_accuracy: 0.8167
Epoch 2/10
318/318 [==============================] - 81s 256ms/step - loss: 0.3574 - accuracy: 0.8785 - val_loss: 0.4425 - val_accuracy: 0.8618
Epoch 3/10
318/318 [==============================] - 81s 255ms/step - loss: 0.2343 - accuracy: 0.9261 - val_loss: 0.4579 - val_accuracy: 0.8547
Epoch 4/10
318/318 [==============================] - 74s 234ms/step - loss: 0.1807 - accuracy: 0.9440 - val_loss: 0.5198 - val_accuracy: 0.8468
Epoch 5/10
318/318 [==============================] - 73s 230ms/step - loss: 0.1528 - accuracy: 0.9523 - val_loss: 0.5695 - val_accuracy: 0.8423
Epoch 6/10
318/318 [==============================] - 74s 233ms/step - loss: 0.1332 - accuracy: 0.9575 - val_loss: 0.6396 - val_accuracy: 0.8379
Epoch 7/10
318/318 [==============================] - 74s 233ms/step - loss: 0.1177 - accuracy: 0.9625 - val_loss: 0.6384 - val_ac

In [24]:
metrcs = lstm_model.evaluate(X_test, y_test)
lstm_loss, lstm_acc = metrcs[0], metrcs[1]
print("LSTM Test Performance")
print("LSTM Loss: " +str(lstm_loss))
print("LSTM accuracy: " + str(lstm_acc))

152/152 [==============================] - 5s 31ms/step - loss: 0.6239 - accuracy: 0.8441
LSTM Test Performance
LSTM Loss: 0.6239235401153564
LSTM accuracy: 0.8440860509872437


In [27]:
#Initializing Convolution neural network
cnn = Sequential()
cnn.add(Embedding(MAX_WORDS, embed_dim, input_length=X.shape[1]))
cnn.add(Conv1D(filters=128, kernel_size=4, padding="same", activation="relu"))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Conv1D(filters=64, kernel_size=4, padding="same", activation="relu"))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dense(3, activation="softmax"))
cnn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
#fitting the model
cnn_history = cnn.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
353/353 [==============================] - 27s 74ms/step - loss: 0.6320 - accuracy: 0.7460
Epoch 2/10
353/353 [==============================] - 26s 72ms/step - loss: 0.3632 - accuracy: 0.8787
Epoch 3/10
353/353 [==============================] - 27s 76ms/step - loss: 0.2174 - accuracy: 0.9273
Epoch 4/10
353/353 [==============================] - 21s 60ms/step - loss: 0.1370 - accuracy: 0.9531
Epoch 5/10
353/353 [==============================] - 21s 59ms/step - loss: 0.1036 - accuracy: 0.9660
Epoch 6/10
353/353 [==============================] - 21s 59ms/step - loss: 0.0803 - accuracy: 0.9739
Epoch 7/10
353/353 [==============================] - 21s 59ms/step - loss: 0.0746 - accuracy: 0.9760
Epoch 8/10
353/353 [==============================] - 22s 61ms/step - loss: 0.0648 - accuracy: 0.9791
Epoch 9/10
353/353 [==============================] - 21s 59ms/step - loss: 0.0564 - accuracy: 0.9817
Epoch 10/10
353/353 [==============================] - 21s 59ms/step - loss: 0.063

In [30]:
cnn_loss, cnn_acc = cnn.evaluate(X_test, y_test)
print("CNN Test Performance")
print("CNN Loss: " + str(cnn_loss))
print("CNN Acc: " + str(cnn_acc))

152/152 [==============================] - 2s 15ms/step - loss: 1.1292 - accuracy: 0.8457
CNN Test Performance
CNN Loss: 1.1292004585266113
CNN Acc: 0.8457402586936951


In [ ]:
#Transformer models: BERT
